# Функции активации и функции потерь

Этот блокнот посвящен изучению ключевых компонентов, определяющих работу нейронных сетей: **функций активации** и **функций потерь**. Мы рассмотрим их свойства, применение и влияние на обучение модели с практическими примерами.

In [ ]:
# Сгружаем датасет

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LeakyReLU
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Загрузка данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Нормализация
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

## Часть 1: Функции активации
Функции активации определяют, как выход каждого нейрона зависит от входных данных. Они добавляют нелинейность, что позволяет модели изучать сложные зависимости.



### Основные функции активации
1. **ReLU (Rectified Linear Unit)**  
   - Одна из самых популярных функций. Возвращает ноль для отрицательных значений и линейное значение для положительных.
   - Пример:
   \begin{equation}
   f(x) = \max(0, x)
   \end{equation}

2. **LeakyReLU**  
   - Модификация ReLU, позволяющая небольшие отрицательные значения. Решает проблему "затухания нейронов".
   - Пример:
   \begin{equation}
   f(x) =
   \begin{cases}
   x, & \text{если } x > 0 \\
   \alpha x, & \text{если } x \leq 0
   \end{cases}
   \end{equation}

3. **Sigmoid**  
   - Используется в выходном слое для бинарной классификации. Преобразует вход в диапазон (0, 1).
   - Пример:
   \begin{equation}
   f(x) = \frac{1}{1 + e^{-x}}
   \end{equation}

4. **Tanh**  
   - Симметричный аналог `Sigmoid`. Возвращает значения в диапазоне (-1, 1).
   - Пример:
   \begin{equation}
   f(x) = \tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}
   \end{equation}

5. **Softplus**  
   - Сглаженный аналог ReLU. Возвращает положительные значения, но без резкого перехода.
   - Пример:
   \begin{equation}
   f(x) = \ln(1 + e^x)
   \end{equation}

6. **ELU (Exponential Linear Unit)**  
   - Включает экспоненциальную часть для обработки отрицательных значений.
   - Пример:
   \begin{equation}
   f(x) =
   \begin{cases}
   x, & \text{если } x > 0 \\
   \alpha (e^x - 1), & \text{если } x \leq 0
   \end{cases}
   \end{equation}

---

### Пример работы с функциями активации
Создадим модель с разными функциями активации и оценим их влияние.

```python
# Функция для обучения модели
def train_model(activation_layer, name):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128),
        activation_layer,
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=0, validation_data=(x_test, y_test))
    _, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Функция активации: {name}, Точность на тесте: {test_accuracy:.4f}")

# Обучение моделей
train_model(Dense(128, activation='relu'), "ReLU")
train_model(Dense(128, activation='sigmoid'), "Sigmoid")
train_model(Dense(128, activation='tanh'), "Tanh")
train_model(Dense(128, activation='softplus'), "Softplus")
train_model(Dense(128, activation='elu'), "ELU")
```

### Задания:
1. Протестируйте модели с разными функциями активации
2. Добавьте другие функции активации (из презентации с прошлой пары) и протестируйте их
3. Измените архитектуру модели (добавьте слои или кол-во нейронов) и протестируйте: возможно, некоторые функции активации поведут себя лучше при другой архитектуре

In [ ]:
# Code here

## Часть 2: Функции потерь
Функции потерь измеряют разницу между предсказаниями модели и истинными значениями, определяя, насколько хорошо обучается модель.



### Основные функции потерь
1. **Mean Squared Error (MSE)**  
   - Для задач регрессии. Измеряет среднеквадратичное отклонение между предсказаниями и истинными значениями.
   - Пример:
   \begin{equation}
   \text{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2
   \end{equation}

2. **Mean Absolute Error (MAE)**  
   - Устойчив к выбросам. Измеряет среднее абсолютное отклонение.
   - Пример:
   \begin{equation}
   \text{MAE} = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y}_i|
   \end{equation}

3. **Categorical Crossentropy**  
   - Для многоклассовой классификации. Сравнивает распределения истинных и предсказанных вероятностей.
   - Пример:
   \begin{equation}
   \text{CCE} = -\sum_{i} y_i \log(\hat{y}_i)
   \end{equation}

4. **Binary Crossentropy**  
   - Для бинарной классификации. Использует логарифмы вероятностей для оценки ошибок.
   - Пример:
   \begin{equation}
   \text{BCE} = -\frac{1}{n} \sum_{i=1}^n \left[ y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i) \right]
   \end{equation}

5. **Huber Loss**  
   - Компромисс между MSE и MAE. Устойчив к выбросам.
   - Пример:
   \begin{equation}
   L_\delta(a) =
   \begin{cases}
   \frac{1}{2}a^2, & \text{если } |a| \leq \delta \\
   \delta(|a| - \frac{1}{2}\delta), & \text{иначе}
   \end{cases}
   \end{equation}

6. **Log-Cosh Loss**  
   - Сглаженный аналог MAE. Хорошо справляется с выбросами.
   - Пример:
   \begin{equation}
   L(y, \hat{y}) = \sum \log(\cosh(\hat{y} - y))
   \end{equation}

---

### Пример работы с функциями потерь
#### Классификация
```python
def train_loss_classification(loss_fn, name):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=0, validation_data=(x_test, y_test))
    _, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Функция потерь: {name}, Точность на тесте: {test_accuracy:.4f}")

train_loss_classification('categorical_crossentropy', "Categorical Crossentropy")
train_loss_classification('mse', "Mean Squared Error")
```

#### Регрессия (будем использовать случайные данные)
```python
import numpy as np
x = np.linspace(-1, 1, 100)
y = x**2 + np.random.normal(0, 0.1, x.shape)

x_train, x_test = x[:80], x[80:]
y_train, y_test = y[:80], y[80:]

def train_loss_regression(loss_fn, name):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(1,)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss=loss_fn, metrics=['mae'])
    model.fit(x_train, y_train, epochs=50, verbose=0, validation_data=(x_test, y_test))
    test_loss = model.evaluate(x_test, y_test, verbose=0)
    print(f"Функция потерь: {name}, Потери на тесте: {test_loss[0]:.4f}")

train_loss_regression('mse', "Mean Squared Error")
train_loss_regression('mae', "Mean Absolute Error")
train_loss_regression(tf.keras.losses.Huber(), "Huber Loss")
train_loss_regression(tf.keras.losses.LogCosh(), "Log-Cosh Loss")
```

### Задания:
1. Протестируйте модели с разными функциями потерь
2. Добавьте другие функции потерь (из презентации с прошлой пары)
3. Измените архитектуру модели (добавьте слои или кол-во нейронов) и протестируйте: возможно, некоторые функции потерь поведут себя лучше при другой архитектуре

In [ ]:
# Code here